In [2]:
import csv
import gurobipy as gp
from gurobipy import GRB
import random
import math
import random
from collections import deque



### Błazej: Optimization having chosen initial districts using greedy

### Błazej: Optimization having chosen initial districts using greedy approach and simulated annealing 

In [3]:

# _____________________________________ Constants _____________________________________
STATE = "TX"
THRESHOLD_PCT = 0.1

# _____________________________________ Load the data _____________________________________
distances = []
with open(f"data/{STATE}/{STATE}_distances.csv", "r", encoding="utf-8") as file:
    reader = csv.reader(file)
    next(reader, None)  # Skip the header
    distances = [list(map(int, row[1:])) for row in reader]

nodes_count = len(distances)
#print(f"Distances:\n{distances}")
#print(f"Nodes count: {nodes_count}")

adjacency_matrix = [[0 for _ in range(nodes_count)] for _ in range(nodes_count)]
districts_count = 4  # This value might be read from your data, hardcoding here for simplicity

with open(f"data/{STATE}/{STATE}.dimacs", "r", encoding="utf-8") as file:
    for line in file:
        if line.startswith("e"):
            _, node1, node2 = line.split()
            adjacency_matrix[int(node1) - 1][int(node2) - 1] = 1
            adjacency_matrix[int(node2) - 1][int(node1) - 1] = 1

#print(f"Adjacency matrix:\n{adjacency_matrix}")
#print(f"Districts count: {districts_count}")

population = []
total_population = 0
with open(f"data/{STATE}/{STATE}.population", "r", encoding="utf-8") as file:
    total_population = int(file.readline().split()[-1])
    for line in file:
        population.append(int(line.split()[1]))

#print(f"Population vector: {population}")
#print(f"Total population: {total_population}")

In [96]:

# _____________________________________ Heuristic for Initial Solution _____________________________________
def create_contiguous_districts(nodes_count, adjacency_matrix, districts_count):
    # Initialize districts
    districts = [[] for _ in range(districts_count)]
    nodes = set(range(nodes_count))

    # Randomly choose seed nodes for each district ensuring they are not the same
    seeds = random.sample(nodes, districts_count)
    for i, seed in enumerate(seeds):
        districts[i].append(seed)
        nodes.remove(seed)

    # Expand each district by adding adjacent nodes until all nodes are assigned
    while nodes:
        for district in districts:
            if nodes:
                # Find all nodes that are adjacent to current district and still unassigned
                adjacent_nodes = set()
                for node in district:
                    for adj, is_connected in enumerate(adjacency_matrix[node]):
                        if is_connected and adj in nodes:
                            adjacent_nodes.add(adj)

                # If there are adjacent nodes, add one to the district
                if adjacent_nodes:
                    new_node = adjacent_nodes.pop()
                    district.append(new_node)
                    nodes.remove(new_node)

                # If there are no adjacent nodes, this means we cannot expand this district further
                # This scenario might require a more sophisticated method to handle

    return districts

# Connectivity check using DFS
def is_connected(district, adjacency_matrix):
    visited = [False] * len(adjacency_matrix)
    
    def dfs(node):
        visited[node] = True
        for i, connected in enumerate(adjacency_matrix[node]):
            if connected and i in district and not visited[i]:
                dfs(i)
                
    dfs(district[0])
    
    return all(visited[node] for node in district)

def get_neighbor(solution, adjacency_matrix, population, total_population, districts_count, THRESHOLD_PCT):
    neighbor = [list(district) for district in solution]
    
    # Calculate the ideal population and allowable deviation
    ideal_population = total_population / districts_count
    lower_bound = ideal_population * (1 - THRESHOLD_PCT)
    upper_bound = ideal_population * (1 + THRESHOLD_PCT)
    
    attempts = 0
    while attempts < 1000:
        # Pick two random districts
        district_a, district_b = random.sample(range(districts_count), 2)
        
        # Calculate the current population of the districts
        pop_a = sum(population[node] for node in neighbor[district_a])
        pop_b = sum(population[node] for node in neighbor[district_b])
        
        # Find border nodes that are eligible for swapping
        border_nodes_a = [node for node in neighbor[district_a] if any(adjacency_matrix[node][other] for other in neighbor[district_b])]
        border_nodes_b = [node for node in neighbor[district_b] if any(adjacency_matrix[node][other] for other in neighbor[district_a])]
        
        # Continue if no border nodes are found
        if not border_nodes_a or not border_nodes_b:
            attempts += 1
            continue
        
        # Find a swap that improves the population balance
        found_valid_swap = False
        for node_a in border_nodes_a:
            for node_b in border_nodes_b:
                # Calculate the new populations after the swap
                new_pop_a = pop_a - population[node_a] + population[node_b]
                new_pop_b = pop_b - population[node_b] + population[node_a]
                
                # Only consider the swap if it brings populations closer to the target range
                if (lower_bound <= new_pop_a <= upper_bound) and (lower_bound <= new_pop_b <= upper_bound):
                    # Check if the swap maintains contiguity
                    temp_neighbor = [list(d) for d in neighbor]
                    temp_neighbor[district_a].remove(node_a)
                    temp_neighbor[district_b].remove(node_b)
                    temp_neighbor[district_a].append(node_b)
                    temp_neighbor[district_b].append(node_a)
                    if is_connected(temp_neighbor[district_a], adjacency_matrix) and is_connected(temp_neighbor[district_b], adjacency_matrix):
                        # Perform the swap
                        neighbor[district_a].remove(node_a)
                        neighbor[district_b].remove(node_b)
                        neighbor[district_a].append(node_b)
                        neighbor[district_b].append(node_a)
                        found_valid_swap = True
                        break
            
            if found_valid_swap:
                break
        
        if found_valid_swap:
            return neighbor
        
        attempts += 1
    
    return solution

def calculate_population_score(solution, population, total_population, districts_count):
    ideal_population = total_population / districts_count
    min_population = ideal_population * 0.8
    max_population = ideal_population * 1.2
    population_score = 0
    
    for district in solution:
        district_population = sum(population[node] for node in district)
        # Check if the district population is within the acceptable range
        if district_population < min_population or district_population > max_population:
            # Apply a heavy penalty if outside of the range
            population_score += (district_population - ideal_population) ** 2
        else:
            # Apply a smaller penalty if within the range
            population_score += ((district_population - ideal_population) ** 2) * 0.1
    
    population_score /= districts_count
    return population_score

def calculate_compactness(solution, distances):
    compactness_score = 0
    for district in solution:
        # Calculate the pairwise distances for all nodes within each district
        district_distances = [distances[node][other_node] for node in district for other_node in district if node != other_node]
        compactness_score += sum(district_distances)
    # Invert the compactness score so that lower scores are better
    max_distance = max(max(row) for row in distances)
    max_score = max_distance * len(distances) * len(distances)
    compactness_score = max_score - compactness_score
    return compactness_score

def calculate_objective(solution, distances, population, total_population, districts_count):
    population_score = calculate_population_score(solution, population, total_population, districts_count)
    compactness_score = calculate_compactness(solution, distances)
    
    weight_population = 50
    weight_compactness = 1
    
    # Combine scores into a single objective value
    objective_value = (weight_population * population_score) + (weight_compactness * compactness_score)
    return objective_value

def get_initial_contiguous_solution(solution, adjacency_matrix):
    contiguous_solution = []
    for district in solution:
        if is_connected(district, adjacency_matrix):
            contiguous_solution.append(district)
        else:
            # Handle non-contiguous districts, e.g., by merging with adjacent ones
            pass  # Your logic to make the district contiguous
    return contiguous_solution

# Simulated annealing main function
def simulated_annealing(initial_solution, distances, population, adjacency_matrix, total_population, districts_count, THRESHOLD_PCT):
    temp = 10000000
    alpha = 0.95
    current_solution = initial_solution
    current_objective = calculate_objective(current_solution, distances, population, total_population, districts_count)

    while temp > 1:
        # Generate a neighboring solution
        neighbor_solution = get_neighbor(current_solution, adjacency_matrix, population, total_population, districts_count, THRESHOLD_PCT)
        
        # Calculate objective for the neighbor solution
        neighbor_objective = calculate_objective(current_solution, distances, population, total_population, districts_count)

        
        # Ensure the neighbor solution is contiguous and compact
        # Check contiguity for each district in the neighbor solution
        all_districts_contiguous = all(is_connected(district, adjacency_matrix) for district in neighbor_solution)
        
        # Decide whether to accept the neighbor
        if all_districts_contiguous and (neighbor_objective < current_objective or
                math.exp((current_objective - neighbor_objective) / temp) > random.random()):
            current_solution = neighbor_solution
            current_objective = neighbor_objective
        
        # Cool down
        temp *= alpha

    # Make sure the final solution is contiguous, just in case
    final_solution = [district for district in current_solution if is_connected(district, adjacency_matrix)]
    
    return final_solution

In [97]:
sa_initial_solution = create_contiguous_districts(nodes_count, adjacency_matrix, districts_count)
sa_solution = simulated_annealing(sa_initial_solution, distances, population, adjacency_matrix, total_population, districts_count, THRESHOLD_PCT)

/var/folders/44/c61kn_t5793fw4b0n9q5kfn40000gn/T/ipykernel_63541/2591309133.py:8: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  seeds = random.sample(nodes, districts_count)


### Check the population constraint

In [37]:
def print_district_populations(solution, population):
    district_populations = []  
    for district in solution:
        district_population = sum(population[node] for node in district)
        district_populations.append(district_population)
    return district_populations


In [95]:
district_populations = print_district_populations(sa_solution, population)
for i, pop in enumerate(district_populations, start=1):
    print(f"District {i} Total Population: {pop}")


District 1 Total Population: 6628484
District 2 Total Population: 2494659
District 3 Total Population: 13433405
District 4 Total Population: 6588957


### Check for disconnected nodes

In [38]:
def dfs(node, visited, adjacency_matrix, district_nodes):
    visited[node] = True
    for i, connected in enumerate(adjacency_matrix[node]):
        if connected and i in district_nodes and not visited[i]:
            dfs(i, visited, adjacency_matrix, district_nodes)

def find_disconnected_nodes(districts, adjacency_matrix):
    disconnected_nodes = []
    for district in districts:
        visited = [False for _ in range(len(adjacency_matrix))]
        # Start DFS from the first node in the district
        district_nodes = set(district)
        start_node = district[0]
        dfs(start_node, visited, adjacency_matrix, district_nodes)

        # Find all nodes in the district that were not visited
        district_disconnected_nodes = [node for node in district if not visited[node]]
        disconnected_nodes.append(district_disconnected_nodes)
    return disconnected_nodes

In [91]:
disconnected_nodes_per_district = find_disconnected_nodes(sa_solution, adjacency_matrix)
for i, disconnected_nodes in enumerate(disconnected_nodes_per_district, 1):
    if disconnected_nodes:
        print(f"District {i} has disconnected nodes: {disconnected_nodes}")
    else:
        print(f"District {i} is fully connected.")


District 1 is fully connected.
District 2 is fully connected.
District 3 is fully connected.
District 4 is fully connected.


#### This method is depends hardly on the initial solution!